In [18]:
import os
import requests
from typing import List
from dotenv import load_dotenv
import anthropic
import gradio as gr


load_dotenv()

api_key = os.getenv('ANTHROPIC_API_KEY').strip()
if not api_key.startswith('sk-ant-'):
    print(api_key)
    raise ValueError("API key appears to be in incorrect format. Should start with 'sk-ant-'")

system_message = api_key = os.getenv('SYSTEM_MESSAGE')
if not system_message:
    raise ValueError(
        "SYSTEM_MESSAGE not found in environment variables. "
        "This is required for the chatbot to understand your background and expertise. "
        "Please add a SYSTEM_MESSAGE entry to your .env file."
    )
    


client = anthropic.Anthropic(api_key=api_key)

In [19]:
system_message = "My name is Adam Lindberg."
system_message += "You are my helpful assistant towards potential employers. My area is software development and engineering"
system_message += "Don't answer to questions you don't have an answer to."

In [20]:
def stream_claude_3_7_sonnet(prompt):
    result = client.messages.stream(
        model="claude-3-7-sonnet-20250219",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    
    response = ""
    
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [23]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    
    with gr.Row():
        submit_btn = gr.Button("Submit")
        clear = gr.Button("Clear")
    
    def user(user_message, history):
        return "", history + [[user_message, None]]
    
    def bot(history):
        user_message = history[-1][0]
        history[-1][1] = ""
        for response in stream_claude_3_7_sonnet(user_message):
            history[-1][1] = response
            yield history
    
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    
    submit_btn.click(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    
    clear.click(lambda: None, None, chatbot, queue=False)

demo.queue()
demo.launch()

/var/folders/w2/nyq_87_d7l5dzyd9hf6z84lw0000gn/T/ipykernel_36881/2418887786.py:2: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


In [22]:
# theme = gr.themes.Monochrome()


# view = gr.Interface(
#     fn=stream_claude3_5Haiku,
#     theme=theme,
#     inputs=[gr.Textbox(label="Your message:")],
#     outputs=[gr.Markdown(label="Response:")],
#     flagging_mode="never"
# )
# view.launch()